# Контекст

Задачи на которых обучается BERT

- Маскированное языковое моделирование (MLM)
- Прогнозирование следующего предложения (NSP)
- анализ тональности
- классификация предложений


- В нашем случаем воспользуемся BERT-base и обучим ее для предсказания [MASK] токена

---

Токенизатор: WordPiece

Пример токенизации слова, которого нет словаре: ['телекоммуника', '##ция'] 
Слова 'телекоммуникация' нет в словаре RU, и токенизатор разбивает его на уже известные токены

---

Для русского языка использовать следующий код токенизатора:

from transformers import AutoModel, AutoTokenizer
tokenizerRU = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
tokenizerRU.vocab

---


In [18]:
import pandas as pd
import warnings

from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizerFast,\
                         LineByLineTextDataset,\
                         DataCollatorForLanguageModeling,\
                         TrainingArguments,\
                         BertConfig,\
                         BertForMaskedLM,\
                         Trainer


warnings.filterwarnings("ignore")

# Чтение файла

In [2]:
imdb_df = pd.read_csv('IMDB_Dataset.csv')
imdb_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
# Записываем отзывы в текстовый файл
rewiews = imdb_df.review.to_string(index = None)

with open('corpus.txt', 'w') as file:
    file.writelines(rewiews)

# Токенизатор BertWordPiece (Создание - сохранение - загрузка словаря)

In [4]:
# создание словаря токенов
bert_word_piece_tokenizer = BertWordPieceTokenizer()
bert_word_piece_tokenizer.train('corpus.txt')

In [5]:
len(bert_word_piece_tokenizer.get_vocab())

18308

In [6]:
# сохранение
bert_word_piece_tokenizer.save_model('tokenizer')

['tokenizer\\vocab.txt']

In [7]:
# загрузка
tokenizer = BertWordPieceTokenizer.from_file('tokenizer/vocab.txt')
tokenizer

Tokenizer(vocabulary_size=18308, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

# Пример использования токенизатора

In [8]:
text = 'Hi, tell me please, what`s your name ?'

text_encode = tokenizer.encode(text)
text_encode.tokens

['[CLS]',
 'hi',
 ',',
 'tell',
 'me',
 'please',
 ',',
 'what',
 '`',
 's',
 'your',
 'name',
 '?',
 '[SEP]']

# Обучение собственной модели BERT

### BertTokenizerFast

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('tokenizer')
tokenizer

BertTokenizerFast(name_or_path='tokenizer', vocab_size=18308, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

### LineByLineTextDataset

In [10]:
dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = 'corpus.txt',
    block_size = 128  
)

dataset

### Систематизатор данных (DataCollator)

In [11]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True,
    mlm_probability = 0.15 # вероятность маскирование токена
)

### Training arguments

In [12]:
training_args = TrainingArguments(
    output_dir = 'BERT',
    overwrite_output_dir = True,
    num_train_epochs = 1,
    per_device_train_batch_size = 128
)

### Модель BERT (конфигуряция по умолчанию)

In [13]:
bert = BertForMaskedLM(BertConfig())

### Объект тренера

In [14]:
trainer = Trainer(
    model = bert,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset
)

### обучение

In [15]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: oudelexsus2010. Use `wandb login --relogin` to force relogin


  0%|          | 0/391 [00:00<?, ?it/s]

{'train_runtime': 126.8403, 'train_samples_per_second': 394.37, 'train_steps_per_second': 3.083, 'train_loss': 5.413248906050192, 'epoch': 1.0}


TrainOutput(global_step=391, training_loss=5.413248906050192, metrics={'train_runtime': 126.8403, 'train_samples_per_second': 394.37, 'train_steps_per_second': 3.083, 'train_loss': 5.413248906050192, 'epoch': 1.0})

### сохранение

In [16]:
trainer.save_model('MyBERT')